In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

Clone OpenAI-Gym

In [ ]:
!git clone https://github.com/openai/gym.git

Installing requirements for the Gym

In [ ]:
%cd gym
!pip3 install -e .

Checking whether the installation was successful by listing all the available environments

In [ ]:
from gym import envs
envids = [spec.id for spec in envs.registry.all()]
for envid in sorted(envids):
    print(envid)

In [ ]:
%cd ..

/content


Cloning the DT Gym and our repository

In [ ]:
!git clone https://github.com/Comicboy/deep-rl-for-the-duckietown-aido_lf-challenge

In [ ]:
!git clone https://github.com/duckietown/gym-duckietown.git

In [ ]:
%cd gym-duckietown

/content/gym-duckietown


Going back to an older commit because it is possible only with this version to register the environments

In [ ]:
!git checkout dcf8dd3

In [ ]:
!pip install -e .

In [ ]:
%cd ..

/content


Copy our map into the maps folder

In [ ]:
!cp deep-rl-for-the-duckietown-aido_lf-challenge/my.yaml gym-duckietown/gym_duckietown/maps

Import Gym

In [1]:
import gym

Import the DT Gym

In [ ]:
%cd gym-duckietown/

/content/gym-duckietown


In [2]:
import gym_duckietown.envs.duckietown_env

INFO:gym-duckietown:gym-duckietown 2018.10.1

INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-my-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0


Get the StableBaselines

In [ ]:
!apt-get update

In [ ]:
!apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev

In [ ]:
!pip install stable-baselines[mpi]

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
import numpy as np
from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy

Get Optuna

In [ ]:
!pip install optuna

In [4]:
import optuna

Virtual display is necessary for training

In [ ]:
!apt update

In [ ]:
!apt-get install python-opengl -y
!apt-get install xvfb -y

In [ ]:
!pip install pyvirtualdisplay
!pip install piglet

In [8]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

A different way of creating a virtual display

In [ ]:
!apt update

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

In [ ]:
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

Check if the DT environments follow the Gym interface

In [ ]:
from gym_duckietown.simulator import Simulator

In [ ]:
from stable_baselines.common.env_checker import check_env
check_env(Simulator())

In [ ]:
from gym_duckietown.envs.duckietown_env import DuckietownEnv

In [ ]:
env.reset()

In [ ]:
env.render()

Defining the parameters for hyperparameter optimization.

The ranges for the parameters are based on this article:
https://medium.com/aureliantactics/ppo-hyperparameters-and-ranges-6fc2d29bccbe

The suggestions' distributions where given intuitively.


In [ ]:
def ppo_hyperopt(trial):

  return {
      #'n_steps': int(trial.suggest_int('n_steps', 32, 5000)),
      #'nminibatches': trial.suggest_int('nminibatches', 4, 4096),
      'gamma': trial.suggest_uniform('gamma', 0.8, 0.9997),
      'learning_rate': trial.suggest_loguniform('learning_rate', 5e-6, 0.003),
      'ent_coef': trial.suggest_uniform('ent_coef', 0, 0.01),
      'noptepochs': int(trial.suggest_int('noptepochs', 3, 30)),
      'vf_coef': trial.suggest_uniform('vf_coef', 0.5, 1),
      'lam': trial.suggest_uniform('lam', 0.9, 1),
      'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.3),
      #'max_grad_norm': trial.suggest_loguniform('max_grad_norm', 0.1, 0.9),
  }

Own trials.

In [ ]:
def ppo_hyperopt(trial):

  return {
      #'n_steps': int(trial.suggest_discrete_uniform('n_steps', 16, 1024, 4)),
      'gamma': trial.suggest_loguniform('gamma', 0.8, 0.95),
      'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-2),
      'ent_coef': trial.suggest_loguniform('ent_coef', 1e-3, 1e-1),
      'noptepochs': int(trial.suggest_int('noptepochs', 3, 10)),
      #'vf_coef': trial.suggest_loguniform('vf_coef', 0.1, 0.9),
      #'max_grad_norm': trial.suggest_loguniform('max_grad_norm', 0.1, 0.9),
      #'lam': trial.suggest_loguniform('lam', 0.8, 0.99),
      #'cliprange': trial.suggest_loguniform('cliprange', 0.15, 0.25)
  }

In [ ]:
def optimize_agent(trial):
  model_params = ppo_hyperopt(trial)
  env = gym.make('Duckietown-my-v0')
  model = PPO2('CnnPolicy', env, verbose=1, **model_params)

  # Learning
  model.learn(10000)

  # Evaluation
  rewards = []
  n_episodes = 0
  reward_sum = 0.0

  obsv = env.reset()
  while n_episodes < 10:
    action, _ = model.predict(obsv)
    obsv, reward, done, info = env.step(action)
    reward_sum = reward_sum + reward

    if done:
      rewards.append(reward_sum)
      reward_sum = 0.0
      n_episodes = n_episodes + 1
      obsv = env.reset()

  last_reward = np.mean(rewards)
  trial.report(-1 * last_reward, n_episodes)

  if trial.should_prune():
    raise optuna.TrialPruned()

  return -1 * last_reward

#env.close()

#mean_reward, std_reward = evaluate_policy(model = model, env = env, n_eval_episodes = 10)
#return mean_reward

In [ ]:
from optuna.samplers import TPESampler
sampler = TPESampler(seed = 3)

In [ ]:
study = optuna.create_study(pruner = optuna.pruners.MedianPruner(n_startup_trials = 3, n_warmup_steps = 1000), sampler = sampler)

[I 2020-12-11 21:20:21,092] A new study created in memory with name: no-name-c12eac79-2901-4297-aad4-a62871c70ef5


In [ ]:
study.optimize(optimize_agent, n_trials = 10, gc_after_trial = True, catch = (float('nan'), ))
print('study.best_params')

DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:loading map file "/content/gym-duckietown/gym_duckietown/maps/my.yaml"
DEBUG:gym-duckietown:loading mesh "/content/gym-duckietown/gym_duckietown/meshes/duckiebot.obj"
DEBUG:gym-duckietown:loading materials from "/content/gym-duckietown/gym_duckietown/meshes/duckiebot.mtl"
DEBUG:gym-duckietown:loading mesh "/content/gym-duckietown/gym_duckietown/meshes/tree.obj"
DEBUG:gym-duckietown:loading materials from "/content/gym-duckietown/gym_duckietown/meshes/tree.mtl"
DEBUG:gym-duckietown:loading mesh "/content/gym-duckietown/gym_duckietown/meshes/duckie.obj"
DEBUG:gym-duckietown:loading texture "/content/gym-duckietown/gym_duckietown/textures/duckie.png"
DEBUG:gym-duckietown:loading texture "/content/gym-duckietown/gym_duckietown/textures/grass_1.png"
DEBUG:gym-duckietown:loading texture "/content/gym-duckietown/gym_duckietown/textures/gras

Wrapping the env in a DummyVecEnv.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



INFO:gym-duckietown:Starting at [0.72427834 0.         0.9495995 ] 4.691989355719424
INFO:gym-duckietown:Pos: [0.72427834 0.         0.9495995 ] angle 4.691989355719424
INFO:gym-duckietown:Pos: [0.72429647 0.         0.95586274] angle 4.73857682369054
INFO:gym-duckietown:Pos: [0.72428024 0.         0.95506412] angle 4.72682493423002
INFO:gym-duckietown:Pos: [0.72370282 0.         0.91506829] angle 4.72682493423002
INFO:gym-duckietown:Pos: [0.72312541 0.         0.87507246] angle 4.72682493423002
INFO:gym-duckietown:Pos: [0.72254799 0.         0.83507662] angle 4.72682493423002
INFO:gym-duckietown:Pos: [0.72289916 0.         0.8443798 ] angle 4.773412402201136
INFO:gym-duckietown:Pos: [0.72307451 0.         0.84797281] angle 4.748896682750725
INFO:gym-duckietown:Pos: [0.72161453 0.         0.80799946] angle 4.748896682750725
INFO:gym-duckietown:Pos: [0.72180959 0.         0.81125737] a

---------------------------------------
| approxkl           | 1.882831e-05   |
| clipfrac           | 0.0            |
| explained_variance | 0.000895       |
| fps                | 7              |
| n_updates          | 1              |
| policy_entropy     | 2.8378947      |
| policy_loss        | -0.00089195714 |
| serial_timesteps   | 128            |
| time_elapsed       | 3.65e-05       |
| total_timesteps    | 128            |
| value_loss         | 13660.038      |
---------------------------------------


INFO:gym-duckietown:Pos: [1.04122697 0.         6.05736261] angle 1.8922972710115522
INFO:gym-duckietown:Pos: [1.03601422 0.         6.04229655] angle 1.9154960152142786
INFO:gym-duckietown:Pos: [1.02250119 0.         6.00533845] angle 1.927159740581649
INFO:gym-duckietown:Pos: [1.00854646 0.         5.96785159] angle 1.927159740581649
INFO:gym-duckietown:Pos: [1.01260778 0.         5.97802995] angle 1.9737472085527648
INFO:gym-duckietown:Pos: [1.02829316 0.         6.01482627] angle 1.9737472085527648
INFO:gym-duckietown:Pos: [1.01895127 0.         5.99192236] angle 1.94239596235097
INFO:gym-duckietown:Pos: [1.01119942 0.         5.9710686 ] angle 1.910988016068035
INFO:gym-duckietown:Pos: [1.02454613 0.         6.00877623] angle 1.910988016068035
INFO:gym-duckietown:Pos: [1.0293088  0.         6.02313623] angle 1.8710958671742286
INFO:gym-duckietown:Pos: [1.03719682 0.         6.05088014] angle 1.8245083992031128
INFO:gym-duckietown:Pos: [1.0308368 0.        6.0278208] angle 1.855324

...

-------------------------------------
| approxkl           | 0.09661364   |
| clipfrac           | 0.35100445   |
| explained_variance | -0.00291     |
| fps                | 14           |
| n_updates          | 2            |
| policy_entropy     | 2.841771     |
| policy_loss        | -0.049230557 |
| serial_timesteps   | 256          |
| time_elapsed       | 9.49         |
| total_timesteps    | 256          |
| value_loss         | 18328.793    |
-------------------------------------


INFO:gym-duckietown:Pos: [5.63568372 0.         4.13930631] angle 6.156661956063724
INFO:gym-duckietown:Pos: [5.65314603 0.         4.14194222] angle 6.110074488092608
INFO:gym-duckietown:Pos: [5.64527985 0.         4.14049055] angle 6.0913124703967
INFO:gym-duckietown:Pos: [5.68454581 0.         4.14811846] angle 6.0913124703967
INFO:gym-duckietown:Pos: [5.680252   0.         4.14736957] angle 6.129707958184039
INFO:gym-duckietown:Pos: [5.71978182 0.         4.15348459] angle 6.129707958184039
INFO:gym-duckietown:Pos: [5.75013813 0.         4.15770376] angle 6.160455521657011
INFO:gym-duckietown:Pos: [5.78800287 0.         4.16170402] angle 6.195403865531623
INFO:gym-duckietown:Pos: [5.74815688 0.         4.15819727] angle 6.195403865531623
INFO:gym-duckietown:Pos: [5.74269084 0.         4.157651  ] angle 6.171751481804164
INFO:gym-duckietown:Pos: [5.7086151  0.         4.15365295] angle 6.16102994464088
INFO:gym-duckietown:Pos: [5.68301691 0.         4.15084227] angle 6.1866172842398

------------------------------------
| approxkl           | 0.18578729  |
| clipfrac           | 0.5329241   |
| explained_variance | 0.0286      |
| fps                | 14          |
| n_updates          | 3           |
| policy_entropy     | 2.8235893   |
| policy_loss        | -0.12573537 |
| serial_timesteps   | 384         |
| time_elapsed       | 18.3        |
| total_timesteps    | 384         |
| value_loss         | 304.05795   |
------------------------------------


INFO:gym-duckietown:Starting at [5.58573462 0.         3.76242607] 4.503528233519441
INFO:gym-duckietown:Pos: [5.58573462 0.         3.76242607] angle 4.503528233519441
INFO:gym-duckietown:Pos: [5.58534991 0.         3.76443279] angle 4.54242265647398
INFO:gym-duckietown:Pos: [5.57826042 0.         3.8021597 ] angle 4.510855892288774
INFO:gym-duckietown:Pos: [5.57025356 0.         3.84135014] angle 4.510855892288774
INFO:gym-duckietown:Pos: [5.57185765 0.         3.83244599] angle 4.55744336025989
INFO:gym-duckietown:Pos: [5.5656846  0.         3.87196678] angle 4.55744336025989
INFO:gym-duckietown:Pos: [5.55951154 0.         3.91148758] angle 4.55744336025989
INFO:gym-duckietown:Pos: [5.56414131 0.         3.88578824] angle 4.510855892288774
INFO:gym-duckietown:Pos: [5.55929675 0.         3.91161674] angle 4.543097787108281
INFO:gym-duckietown:Pos: [5.5610188  0.         3.89991744] angle 4.589393868477976
INFO:gym-duckietown:Pos: [5.55611139 0.         3.93961527] angle 4.58939386847

study.best_params


Save study

In [ ]:
!pip install joblib

In [ ]:
import joblib

In [ ]:
joblib.dump(study, 'study_ppo.pkl')

['study_ppo.pkl']

Resume study

In [ ]:
study = joblib.load('study_ppo.pkl')
print('Best trial until now:')
print(' Value: ', study.best_trial.value)
print(' Params: ')
for key, value in study.best_trial.params.items():
    print(f'    {key}: {value}')

Best trial until now:
 Value:  894.1757357146546
 Params: 
    gamma: 0.9155994076755679
    learning_rate: 9.562294452806585e-06
    ent_coef: 0.009540675768545583
    noptepochs: 14
    vf_coef: 0.7701563497196793
    lam: 0.928140098695417
    cliprange: 0.23315876987097497


In [ ]:
 env = gym.make('Duckietown-my-v0')

In [ ]:
env.reset()

In [ ]:
model = PPO2('CnnPolicy', env, gamma = 0.9155994076755679, ent_coef = 0.009540675768545583, learning_rate = 9.562294452806585e-06, vf_coef = 0.7701563497196793, lam =  0.928140098695417, noptepochs = 14, cliprange = 0.23315876987097497,   verbose=1).learn(100000)

In [12]:
model.save('ppo_hiperopt')

In [ ]:
evaluate_policy(model = model, env = env, n_eval_episodes = 10)

This is our CNN structure using tensorflow.nn.

In [ ]:
from stable_baselines.common.tf_util import batch_to_seq, seq_to_batch
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc, lstm
from stable_baselines.common.distributions import make_proba_dist_type, CategoricalProbabilityDistribution, \
    MultiCategoricalProbabilityDistribution, DiagGaussianProbabilityDistribution, BernoulliProbabilityDistribution
from stable_baselines.common.input import observation_input

In [ ]:
def my_cnn(scaled_images, **kwargs):

    activ = tf.nn.leaky_relu
    layer_1 = activ(conv(scaled_images, n_filters=64, filter_size=5, stride=2, init_scale=np.sqrt(2), **kwargs))
    layer_2 = activ(conv(layer_1, n_filters=64, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_1 = tf.nn.max_pool2d(layer_2, ksize = 2, strides = 2)
    layer_3 = activ(conv(max_pool_1, n_filters=128, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    layer_4 = activ(conv(layer_3, n_filters=128, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_2= tf.nn.max_pool2d(layer_4, ksize = 2, strides = 2)
    layer_5 = activ(conv(max_pool_2, n_filters=256, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_3= tf.nn.max_pool2d(layer_5, ksize = 2, strides = 2)

    max_pool_3 = conv_to_fc(max_pool_3) # no flatten???

    linear_1 = activ(linear(max_pool_3, n_hidden = 2048, init_scale=np.sqrt(2)))
    linear_2 = activ(linear(linear_1, n_hidden = 2048, init_scale=np.sqrt(2)))
    return activ(linear(layer_3, n_hidden = 3, init_scale=np.sqrt(2)))
